# SB1 Big Data Downloads
An analysis into the corridor analysis data downloads from Streetlight and Replica

In [1]:
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/corridor_study_data/"

#### Read in Replica Data

In [4]:
replica_trips = "replica-bay_bridge-03_06_24-trips_dataset.csv"

In [5]:
r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))

/tmp/ipykernel_156/2643890524.py:1: DtypeWarning: Columns (2,4,5,9,10,11,18,25,26,29,30,35,36,40,43,48,50) have mixed types. Specify dtype option on import or set low_memory=False.
  r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))


In [6]:
r_trips.sample(5)

,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
316049,single_family,53,english,"327 (San Francisco, CA)",9141816078114771967,male,"7 (Tract 327, San Francisco, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,69.0,13927635906260089585,"1 (Tract 4287, Alameda, CA)",single_family,unemployed_under_16_not_in_labor_force,California,Alameda,Does not have work/school location,"4287 (Alameda, CA)",0.0,"7 (Tract 327, San Francisco, CA)",other_travel_mode,two,auto_passenger,home,single_family,NaN,Does not have work/school location,1.0,San Francisco,single_family,not_working,owner,unknown_fuel_type,"327 (San Francisco, CA)",Does not have work/school location,advanced_degree,05:42:00,0.0,not_in_labor_force,work,single_family,Does not have work/school location,24.1,not_attending_school,San Francisco,18329131915304783635,06:35:44
589366,multi_family,29,english,"6041.03 (San Mateo, CA)",9632183769534570447,female,"2 (Tract 6041.03, San Mateo, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,46.0,7705858213998213664,"3 (Tract 4012, Alameda, CA)",non_retail_attraction,remote,California,Alameda,"3 (Tract 6024, San Mateo, CA)","4012 (Alameda, CA)",83424.0,"2 (Tract 6041.03, San Mateo, CA)",worked_from_home,one,auto_passenger,work_from_home,multiple_units,NaN,California,1.0,San Mateo,mixed_use,naics621492,renter,unknown_fuel_type,"6041.03 (San Mateo, CA)","6024 (San Mateo, CA)",some_college,10:44:00,83424.0,employed,recreation,multi_family,San Mateo,22.6,not_attending_school,San Mateo,2062283279317414522,11:13:30
248983,single_family,59,english,"6038.02 (San Mateo, CA)",11049592624520655437,female,"1 (Tract 6038.02, San Mateo, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,51.0,9727666906861881478,"2 (Tract 3580, Contra Costa, CA)",retail,remote,California,Contra Costa,"1 (Tract 6038.02, San Mateo, CA)","3580 (Contra Costa, CA)",207015.0,"1 (Tract 6038.02, San Mateo, CA)",worked_from_home,three_plus,private_auto,home,single_family,NaN,California,3.0,San Mateo,retail,naics62,owner,other_non_bev,"6038.02 (San Mateo, CA)","6038.02 (San Mateo, CA)",some_college,18:24:00,85484.0,employed,shop,single_family,San Mateo,38.6,not_attending_school,San Mateo,4488848831358882611,19:23:48
18698,retail,31,NaN,"110.02 (San Francisco, CA)",,NaN,Visitor (no home location),California,Visitor (no home location),NaN,NaN,NaN,visitor,unknown_vehicle_type,NaN,6990046985316847809,"1 (Tract 4230, Alameda, CA)",retail,NaN,California,Alameda,Visitor (no work/school location),"4230 (Alameda, CA)",NaN,"1 (Tract 110.02, San Francisco, CA)",NaN,NaN,auto_passenger,shop,NaN,NaN,Visitor (no work/school location),NaN,San Francisco,mixed_use,NaN,NaN,unknown_fuel_type,Visitor (no home location),Visitor (no work/school location),NaN,17:31:08,NaN,NaN,eat,mixed_use,Visitor (no work/school location),13.1,NaN,Visitor (no home location),172363010

In [7]:
len(r_trips)

674864

In [30]:
r_trips>>count(_.trip_taker_household_id)>>arrange(-_.n)

,trip_taker_household_id,n
248686,,27897
262322,NaN,16384
110604,8194219563640751815,28
82299,6097771312917788596,27
74914,5552467881171814730,22
...,...,...
262312,9988489366890220188,1
262316,9991342309439330560,1
262317,9992490903429866665,1
262318,9992599661009943006,1


In [32]:
r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)

,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
115980,single_family,8,indo_european,"179.03 (San Francisco, CA)",8194219563640751815,male,"1 (Tract 179.03, San Francisco, CA)",California,California,hispanic_or_latino_origin,NaN,NaN,core,unknown_vehicle_type,28.0,11649742760210152290,"2 (Tract 180, San Francisco, CA)",industrial,in_person,California,San Francisco,"2 (Tract 180, San Francisco, CA)","180 (San Francisco, CA)",1200990.0,"1 (Tract 179.03, San Francisco, CA)",private_auto,three_plus,auto_passenger,home,single_family,NaN,California,10.0,San Francisco,industrial,naics31_33,renter,unknown_fuel_type,"179.03 (San Francisco, CA)","180 (San Francisco, CA)",bachelors_degree,07:26:00,136907.0,employed,work,single_family,San Francisco,4.3,not_attending_school,San Francisco,10066255028734967962,07:34:11
116099,industrial,8,indo_european,"180 (San Francisco, CA)",8194219563640751815,male,"1 (Tract 179.03, San Francisco, CA)",California,California,hispanic_or_latino_origin,NaN,NaN,core,unknown_vehicle_type,28.0,3527929229602445979,"1 (Tract 179.03, San Francisco, CA)",single_family,in_person,California,San Francisco,"2 (Tract 180, San Francisco, CA)","179.03 (San Francisco, CA)",1200990.0,"2 (Tract 180, San Francisco, CA)",private_auto,three_plus,auto_passenger,work,single_family,NaN,California,10.0,San Francisco,single_family,naics31_33,renter,unknown_fuel_type,"179.03 (San Francisco, CA)","180 (San Francisco, CA)",bachelors_degree,10:34:00,136907.0,employed,home,industrial,San Francisco,4.4,not_attending_school,San Francisco,10066255028734967962,10:42:11
257674,retail,20,indo_european,"4028.01 (Alameda, CA)",8194219563640751815,female,"1 (Tract 179.03, San Francisco, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,22.0,17607258221711465814,"1 (Tract 179.03, San Francisco, CA)",single_family,remote,California,San Francisco,"1 (Tract 179.03, San Francisco, CA)","179.03 (San Francisco, CA)",1200990.0,"1 (Tract 4028.01, Alameda, CA)",worked_from_home,three_plus,private_auto,shop,single_family,NaN,California,10.0,Alameda,single_family,naics62,renter,other_non_bev,"179.03 (San Francisco, CA)","179.03 (San Francisco, CA)",bachelors_degree,17:23:00,76878.0,employed,home,retail,San Francisco,8.3,not_attending_school,San Francisco,1325181746552868554,17:43:25
257680,retail,6,indo_european,"615.06 (San Francisco, CA)",8194219563640751815,male,"1 (Tract 179.03, San Francisco, CA)",California,California,white_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,27.0,4258107892431763503,"1 (Tract 179.03, San Francisco, CA)",single_family,remote,California,San Francisco,"1 (Tract 179.03, San Francisco, CA)","179.03 (San Francisco, CA)",1200990.0,"3 (Tract 615.06, San Francisco, CA)",worked_from_home,three_plus,auto_passenger,shop,single_family,NaN,California,10.0,San Francisco,single_family,naics54,renter,unknown_fuel_type,"179.03 (San Francisco, CA)","179.03 (San Francisco, C

#### Read in Streetlight Data

In [8]:
streetlight = "streetlight_bay_bridge_corridor_study_corridor_study.csv"

In [9]:
sl_data = to_snakecase(pd.read_csv(f"{GCS_PATH}{streetlight}"))

In [10]:
sl_data.sample(5)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
1305,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1128174334,San Francisco – Oakland Bay Bridge / 237731428...,Motorway,0.177,yes,43,EAST,no,1: Weekday (M-F),15: 11am (11am-12noon),8505,58,11,67.749,1504.04,1.16,False,66
329,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,Motorway,0.109,yes,220,WEST,no,1: Weekday (M-F),29: 11pm (11pm-12am),1824,65,7,68.596,199.30,1.05,False,74
399,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,Motorway,0.073,yes,37,EAST,no,1: Weekday (M-F),09: Peak AM (6am-10am),24210,59,3,67.267,1761.56,1.14,False,70
644,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1112052563,San Francisco – Oakland Bay Bridge / 236348360...,Motorway,0.054,yes,231,WEST,no,0: All Days (M-Su),14: Mid-Day (10am-4pm),44527,52,3,69.248,2387.11,1.34,True,66
208,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1022959400,San Francisco – Oakland Bay Bridge / 661905446...,Motorway,1.939,yes,40,EAST,no,0: All Days (M-Su),28: 10pm (10pm-11pm),5492,55,128,64.627,10647.30,1.17,False,63


In [11]:
len(sl_data)

1710

In [17]:
sl_data>>count(_.zone_id, _.zone_name)

,zone_id,zone_name,n
0,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,90
1,1020456908,San Francisco – Oakland Bay Bridge / 23874736 / 1,90
2,1022959400,San Francisco – Oakland Bay Bridge / 661905446...,90
3,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,90
4,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,90
5,1050419851,San Francisco – Oakland Bay Bridge / 179235221...,90
6,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,90
7,1112052563,San Francisco – Oakland Bay Bridge / 236348360...,90
8,1112062603,San Francisco – Oakland Bay Bridge / 236348361...,90
9,1114495634,I 80 / 395419317 / 1,90


In [19]:
sl_data>>filter(_.zone_id==1133975975)>>count(_.day_part)

,day_part,n
0,00: All Day (12am-12am),3
1,01: 12am (12am-1am),3
2,02: Early AM (12am-6am),3
3,03: 1am (1am-2am),3
4,04: 2am (2am-3am),3
5,05: 3am (3am-4am),3
6,06: 4am (4am-5am),3
7,07: 5am (5am-6am),3
8,08: 6am (6am-7am),3
9,09: Peak AM (6am-10am),3


* The data here shows that each segment is broken out into hour time slots and then also aggregated to peak time periods. moving forward we will look at the `All Day` for `day_part` and `All Days` for `day_type`

In [24]:
sl_data = sl_data>>filter(_.day_part == '00: All Day (12am-12am)', _.day_type == '0: All Days (M-Su)')

In [27]:
len(sl_data)

19

In [28]:
sl_data>>count(_.zone_id, _.zone_name)

,zone_id,zone_name,n
0,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,1
1,1020456908,San Francisco – Oakland Bay Bridge / 23874736 / 1,1
2,1022959400,San Francisco – Oakland Bay Bridge / 661905446...,1
3,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,1
4,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,1
5,1050419851,San Francisco – Oakland Bay Bridge / 179235221...,1
6,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,1
7,1112052563,San Francisco – Oakland Bay Bridge / 236348360...,1
8,1112062603,San Francisco – Oakland Bay Bridge / 236348361...,1
9,1114495634,I 80 / 395419317 / 1,1


In [25]:
sl_data>>filter(_.zone_id==1133975975)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
1530,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1133975975,San Francisco – Oakland Bay Bridge / 929579737...,Motorway,0.273,yes,215,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),128139,49,23,68.634,35038.3,1.39,True,67
